# Solving an optimization problem with the framework

In [67]:
import metaheuristic_designer as mhd
from metaheuristic_designer import ObjectiveVectorFunc
from metaheuristic_designer.initializers import UniformVectorInitializer
from metaheuristic_designer.algorithms import GeneralAlgorithm
# from metaheuristic_designer.strategies import HillClimb, GA, CRO, BinomialPBIL
from metaheuristic_designer.strategies import *
from metaheuristic_designer.operators import OperatorVector
from metaheuristic_designer.selectionMethods import SurvivorSelection, ParentSelection
import numpy as np

## The objective function
To start, we need to define the function we want to optimize.

This is done by creating a class that inherits from ```ObjectiveFunc```, or in this case ```ObjectiveVectorFunc``` since it provides the framework to create objective functions that take vectors as inputs.

For this example, we will try to optimize the path on a grid with randomly placed treasures that the agent must collect. The higher the number of treasures collected, the higher the reward.

In [28]:
class TresureHunt(ObjectiveVectorFunc):
    """
    Our objective function function.

    Takes an integer-valued vector and outputs the number of treasures collected.
    """

    def __init__(self, n_moves, n_tresures, grid_size=(10, 10), name: str = "Tresure Hunt"):
        self.n_moves = n_moves
        self.n_tresures = n_tresures
        self.grid_size = np.array(grid_size)

        # We define a grid with n_treasures
        grid_flat = np.zeros(grid_size).flatten()
        grid_flat[:n_tresures] = 1
        np.random.shuffle(grid_flat)
        self.grid = grid_flat.reshape(grid_size)

        # We define some extra information for displaying the grid
        self.grid_print_dict = {0: " _", 1: " x", 2: " o", 3: " #"}

        # We specify the size of the vector (n_moves), wether to maximize or minimize (mode),
        # lower limit (low_lim), upper limit (up_lim) and name of the function (name).
        super().__init__(n_moves, mode="max", low_lim=0, up_lim=3, name=name)

    def objective(self, move_vector):
        """
        Go through the specified path and count the amount of treasures collected
        """

        # Set of moves the agent can take, accessed from 0 to 3
        move_set = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])

        # Inital position
        pos = np.array([0, 0])

        # Accumulator of treasures collected
        collected = 0

        # Copy of the grid
        grid_copy = self.grid.copy()

        # Travel the path specified by the vector
        for move in move_vector:
            # Move to the next position
            new_pos = pos + move_set[move]

            # Check if it is on bounds
            if new_pos[0] >= 0 and new_pos[0] < self.grid_size[0] and new_pos[1] >= 0 and new_pos[1] < self.grid_size[1]:
                pos = new_pos

                # Collect the treasure if there is one
                if grid_copy[tuple(pos)] == 1:
                    collected += 1
                    grid_copy[tuple(pos)] = 0

        return collected

    def display(self):
        """
        Display the grid as text.
        """

        grid_str = ""
        for row in self.grid:
            for cell in row:
                grid_str += self.grid_print_dict[cell]
            grid_str += "\n"
        print(grid_str)

    def display_solution(self, move_vector):
        """
        Display the grid with the path taken.
        """

        collected = 0
        move_set = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])
        pos = np.array([0, 0])
        grid_copy = self.grid.copy()

        if grid_copy[tuple(pos)] == 0:
            grid_copy[tuple(pos)] = 2
        elif grid_copy[tuple(pos)] == 1:
            grid_copy[tuple(pos)] = 3

        for move in move_vector:
            new_pos = pos + move_set[move]
            if new_pos[0] >= 0 and new_pos[0] < self.grid_size[0] and new_pos[1] >= 0 and new_pos[1] < self.grid_size[1]:
                pos = new_pos
                if grid_copy[tuple(pos)] == 0:
                    grid_copy[tuple(pos)] = 2
                elif grid_copy[tuple(pos)] == 1:
                    grid_copy[tuple(pos)] = 3

        grid_str = ""
        for row in grid_copy:
            for cell in row:
                grid_str += self.grid_print_dict[cell]
            grid_str += "\n"

        print(grid_str)


In [29]:
# Define the objective function
objfunc = TresureHunt(n_moves=200, n_tresures=40, grid_size=(20, 20))
print("The map we use:")
objfunc.display()

The map we use:
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ _ _
 _ _ _ _ _ _ x _ _ _ _ x _ _ _ _ x _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _
 _ _ x _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ x _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ x _ _ x _ _ _ _ _ _ x _ _
 _ _ _ _ x _ _ _ _ _ _ _ _ _ _ x _ _ _ x
 _ _ _ _ x _ x x x _ _ _ _ _ _ _ _ _ _ _
 _ _ _ x _ _ _ _ _ _ _ _ x _ _ _ x _ _ _
 _ _ _ _ _ _ _ _ x _ _ _ _ _ _ _ _ x _ _
 _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ x _ x _
 _ _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ x _ _ _ _ _ _ _ x _ _ _ _ _ _
 x _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _
 _ _ _ _ _ _ _ _ _ _ x _ _ _ x _ _ x _ x
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ x
 _ _ _ _ _ x _ _ _ _ _ _ _ x _ _ _ _ _ _
 _ _ _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _ _



In [ ]:
params = {"stop_cond": "neval", "neval": 50000, "verbose": True, "v_timer": 2}

# Define a population initialization module
pop_init = UniformVectorInitializer(objfunc.vecsize, objfunc.low_lim, objfunc.up_lim, pop_size=1, dtype=int)

# Define the operators to use
mutate_op = OperatorVector("MutSample", {"distrib": "Uniform", "Low": 0, "Up": 3, "N": 8})

# Instanciate the search strategy
strategy = HillClimb(pop_init, mutate_op)

# Define an Algorithm
algorithm = GeneralAlgorithm(objfunc, strategy, params)

# Optimize the objective function
best_population = algorithm.optimize()
best_solution, best_fitness = best_population.best_solution()

Initializing optimization of Tresure Hunt using HillClimb
---------------------------------------------------------

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 0.0 s
	CPU time Spent:  0.0 s
	Generation: 0
	Best fitness: 7.0
	Evaluations of fitness: 1


Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 2.0 s
	CPU time Spent:  2.0 s
	Generation: 1112
	Best fitness: 12.0
	Evaluations of fitness: 1113


Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 4.0 s
	CPU time Spent:  4.0 s
	Generation: 2214
	Best fitness: 12.0
	Evaluations of fitness: 2215


Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 6.0 s
	CPU time Spent:  6.0 s
	Generation: 3321
	Best fitness: 12.0
	Evaluations of fitness: 3322


Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 8.0 s
	CPU time Spent:  8.0 s
	Generation: 4300
	Best fitness: 12.0
	Evaluations of fitness: 4301


Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 10.01 s
	CPU time Spent:  10.01 s
	G

In [47]:
print(f"Number of treasures obtained: {best_fitness}")
objfunc.display_solution(best_solution)

Number of treasures obtained: 13.0
 o o o _ _ _ _ _ _ _ _ _ _ _ x _ _ _ _ _
 _ _ o _ _ _ x _ _ _ _ x _ _ _ _ x _ _ _
 _ _ o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ o o _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _
 _ o # _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ # o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o o _ _ _ _ # o _ x _ _ _ _ _ _ x _ _
 o o o o # o o o o _ _ _ _ _ _ x _ _ _ x
 o o o _ # o # # # _ _ _ _ _ _ _ _ _ _ _
 _ _ _ x o _ o _ _ _ _ _ x _ _ _ x _ _ _
 _ _ _ _ _ o o _ x _ _ _ _ _ _ _ _ x _ _
 _ _ _ _ _ o _ _ _ _ _ _ x _ _ _ x _ x _
 _ _ _ _ _ o # _ _ _ _ _ _ _ _ _ _ _ _ _
 o o o o o # o _ _ _ _ _ _ x _ _ _ _ _ _
 # _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ _ _
 o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _
 o _ _ _ _ _ _ _ _ _ x _ _ _ x _ _ x _ x
 o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ x
 o o o _ o # o _ _ _ _ _ _ x _ _ _ _ _ _
 o o o o o o o # _ _ _ _ _ _ _ _ _ _ _ _



In [48]:
# Define the parameters of the optimization process
params = {"stop_cond": "neval", "neval": 50000, "verbose": True, "v_timer": 2}
pop_size = 100

# Define a population initialization module
pop_init = UniformVectorInitializer(objfunc.vecsize, objfunc.low_lim, objfunc.up_lim, pop_size=pop_size, dtype=int)

# Define the operators to use
mutate_op = OperatorVector("MutSample", {"distrib": "Uniform", "Low": 0, "Up": 3, "N": 8})
cross_op = OperatorVector("Multipoint")

# Define the parent selection method
parent_sel = ParentSelection("Tournament", {"amount": 3, "p": 1})

# Define the survivor selection method
surv_sel = SurvivorSelection("KeepBest")

# Instanciate the algorithm
strategy = GA(pop_init, mutate_op, cross_op, parent_sel, surv_sel, {"pmut": 0.1, "pcross": 0.8})

# Define a Search method
algorithm = GeneralAlgorithm(objfunc, strategy, params)

# Optimize the objective function
best_population = algorithm.optimize()
best_solution, best_fitness = best_population.best_solution()

Initializing optimization of Tresure Hunt using GA
--------------------------------------------------

Optimizing Tresure Hunt using GA:
	Real time Spent: 0.06 s
	CPU time Spent:  0.06 s
	Generation: 0
	Best fitness: 7.0
	Evaluations of fitness: 100

	diversity: 0.948

Optimizing Tresure Hunt using GA:
	Real time Spent: 2.01 s
	CPU time Spent:  2.01 s
	Generation: 30
	Best fitness: 21.0
	Evaluations of fitness: 2665

	diversity: 1.06

Optimizing Tresure Hunt using GA:
	Real time Spent: 4.04 s
	CPU time Spent:  4.04 s
	Generation: 80
	Best fitness: 21.0
	Evaluations of fitness: 5269

	diversity: 1.05

Optimizing Tresure Hunt using GA:
	Real time Spent: 6.07 s
	CPU time Spent:  6.07 s
	Generation: 132
	Best fitness: 21.0
	Evaluations of fitness: 7859

	diversity: 1.05

Optimizing Tresure Hunt using GA:
	Real time Spent: 8.08 s
	CPU time Spent:  8.08 s
	Generation: 185
	Best fitness: 21.0
	Evaluations of fitness: 10414

	diversity: 1.05

Optimizing Tresure Hunt using GA:
	Real time Spent:

In [49]:
print(f"Number of treasures obtained: {best_fitness}")
objfunc.display_solution(best_solution)

Number of treasures obtained: 21.0
 o _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ _ _
 o _ _ _ _ _ x _ _ _ _ x _ _ _ _ x _ _ _
 o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o _ _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _
 o o # o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o # o o o _ _ _ _ _ o o o _ _ _ _ _ _ _
 o _ o _ o o o # o o # o o o o _ _ x _ _
 _ _ _ _ x o o _ _ o o o o o o # o _ _ x
 _ _ _ _ x _ x x x o _ _ _ o o _ _ _ _ _
 _ _ _ x _ _ _ _ _ _ _ _ # o o o # o _ _
 _ _ _ _ _ _ _ _ x _ _ _ o o o o o # o _
 _ _ _ _ _ _ _ _ _ _ _ _ # o o o # o # o
 _ _ _ _ _ _ x _ _ _ _ _ _ o o o _ _ o o
 _ _ _ _ _ x _ _ _ _ _ _ _ # o _ _ o o o
 x _ _ _ _ _ _ _ _ _ _ _ _ o # _ _ o _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ # _ _
 _ _ _ _ _ _ _ _ _ _ x _ _ _ # o o # o #
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ o o # o o x
 _ _ _ _ _ # o o o o o _ o # o o _ o _ _
 _ _ _ _ _ o _ # o o o o o o _ _ _ _ _ _



In [51]:
# Define the parameters of the optimization process
params = {"stop_cond": "neval", "neval": 50000, "verbose": True, "v_timer": 2}
pop_size = 100

# Define a population initialization module
pop_init = UniformVectorInitializer(objfunc.vecsize, objfunc.low_lim, objfunc.up_lim, pop_size=pop_size, dtype=int)

# Define the operators to use
mutate_op = OperatorVector("MutSample", {"distrib": "Uniform", "Low": 0, "Up": 3, "N": 8})
cross_op = OperatorVector("Multipoint")

# Instanciate the algorithm
cro_params = {"rho": 0.8, "Fb": 0.8, "Fd": 0.3, "Pd": 0.2, "attempts": 3}
strategy = CRO(pop_init, mutate_op, cross_op, cro_params)

# Define a Search method
algorithm = GeneralAlgorithm(objfunc, strategy, params)

# Optimize the objective function
best_population = algorithm.optimize()
best_solution, best_fitness = best_population.best_solution()

Initializing optimization of Tresure Hunt using CRO
---------------------------------------------------

Optimizing Tresure Hunt using CRO:
	Real time Spent: 0.05 s
	CPU time Spent:  0.05 s
	Generation: 0
	Best fitness: 6.0
	Evaluations of fitness: 80


Optimizing Tresure Hunt using CRO:
	Real time Spent: 2.05 s
	CPU time Spent:  2.05 s
	Generation: 37
	Best fitness: 20.0
	Evaluations of fitness: 3040


Optimizing Tresure Hunt using CRO:
	Real time Spent: 4.09 s
	CPU time Spent:  4.09 s
	Generation: 73
	Best fitness: 23.0
	Evaluations of fitness: 5920


Optimizing Tresure Hunt using CRO:
	Real time Spent: 6.13 s
	CPU time Spent:  6.13 s
	Generation: 114
	Best fitness: 23.0
	Evaluations of fitness: 8725


Optimizing Tresure Hunt using CRO:
	Real time Spent: 8.18 s
	CPU time Spent:  8.18 s
	Generation: 158
	Best fitness: 23.0
	Evaluations of fitness: 11555


Optimizing Tresure Hunt using CRO:
	Real time Spent: 10.23 s
	CPU time Spent:  10.23 s
	Generation: 199
	Best fitness: 23.0
	Evalua

In [53]:
print(f"Number of treasures obtained: {best_fitness}")
objfunc.display_solution(best_solution)

Number of treasures obtained: 23.0
 o _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ _ _
 o o _ _ _ _ x _ _ _ _ x _ _ _ _ x _ _ _
 o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o _ _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _
 o o # _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ x o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ o o _ _ _ x _ _ x _ _ _ _ _ _ # o o
 _ _ _ o # o o _ _ _ _ _ _ _ _ # o o o #
 _ _ _ _ x _ # # # o _ _ _ _ _ o o o o o
 _ _ _ x _ _ _ _ _ o o o # o o o # o o o
 _ _ _ _ _ _ _ _ x _ o o o _ _ _ o # o o
 _ _ _ _ _ _ _ _ _ _ _ o x _ _ _ # _ x _
 _ _ _ _ _ o # _ _ _ _ _ _ _ _ _ o _ _ _
 _ _ _ _ _ # o o _ _ _ _ _ # o o o o o _
 x _ _ _ _ o o o o o o o _ o # o o o o _
 _ _ _ _ _ _ _ o _ o o o o o o o o x o o
 _ _ _ _ _ o o o _ _ # o o _ x _ o # o #
 _ _ _ _ _ o _ _ _ _ _ _ _ _ _ _ # o o #
 _ _ _ _ _ # o o _ _ _ _ _ x _ _ _ _ o _
 _ _ _ _ o o o # _ _ _ _ _ _ _ _ _ _ _ _

